In [5]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [103]:
def extrude_column(ovrimg, col, val):
    col = int(col)
    z_count = 0
    z_mid = 0
    for z in range(ovrimg.shape[0]):
        val_count = np.count_nonzero(ovrimg[z,:,col]==val)
        if val_count>0:
            z_count += 1
            pos = np.where(ovrimg[z,:,col]==val)
            first = pos[0][0]
            last = pos[0][-1]
            mid = (first+last)//2
            z_mid += mid
    if z_count > img.shape[0]//2:
        z_mid /= z_count
        ovrimg[:,:mid,col]=0
        ovrimg[:,mid:,col]=val
    else:
        ovrimg[:,:,col]=0
    
def extrude_annotations(pathname):
    data_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.mha"))
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.interpolated-overlay.mha"))
    
    num_overlays = len(data_overlay_paths)
    for i in range(num_overlays):
        ovrimg = itk.imread(data_overlay_paths[i], itk.UC)
        print("Processing", data_overlay_paths[i])
        for c in range(ovrimg.shape[2]):
            nos = np.count_nonzero(ovrimg[:,:,c]==1)
            rib = np.count_nonzero(ovrimg[:,:,c]==2)
            sli = np.count_nonzero(ovrimg[:,:,c]==3)
            if nos>0 and rib==0 and sli==0:
                extrude_column(ovrimg,c,1)
            elif nos==0 and rib>0 and sli==0:
                extrude_column(ovrimg,c,2)
            elif nos==0 and rib==0 and sli>0:
                extrude_column(ovrimg,c,3)
            else:
                ovrimg[:,:,c]=0
                
        img = itk.imread(data_paths[i], itk.F)
        imMath = ttk.ImageMath.New(img)
        imMath.Threshold(0,50,0,1)
        imMath.Dilate(10,1,0)
        img = imMath.GetOutputUChar()
        connComp = ttk.SegmentConnectedComponents.New(img)
        connComp.SetMinimumVolume(1000000)
        connComp.Update()
        img_mask = itk.GetArrayFromImage(connComp.GetOutput())
        img_mask = np.where(img_mask>0,1,0)
        keep = img_mask * ovrimg
        keep = np.where(keep==0,keep+2,keep)
        #itkkeep = itk.GetImageFromArray(keep.astype(np.ubyte))
        #itk.imwrite(itkkeep, "test.mha", compression=True)
        ovrimg = itk.GetArrayFromImage(ovrimg)
        for val in [1,3]:
            ovrimg = np.where(ovrimg==val,keep,ovrimg)
            #itkimg = itk.GetImageFromArray(ovrimg.astype(np.ubyte))
            #itk.imwrite(itkimg, "test"+str(val)+".mha", compression=True)
        (filepath,filename) = os.path.split(data_overlay_paths[i])
        new_name = filepath+"/"+filename.split('.')[0]+'.extruded-overlay.mha'
        print("Writing", new_name)
        ovrimg = itk.GetImageFromArray(ovrimg.astype(np.ubyte))
        itk.imwrite(ovrimg, new_name, compression=True)

In [ ]:
#pathname = "../../Data/VFoldData/BAMC-PTXNoSliding"
pathname = "../../Data/VFoldData/BAMC-PTXSliding"
extrude_annotations(pathname)

Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_73815992352100_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_73815992352100_clean.extruded-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_74132233134844_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_74132233134844_clean.extruded-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10705997566592_CLEAN.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10705997566592_CLEAN.extruded-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.extruded-overlay.mha
Processing ../../Data/VFoldD

In [ ]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
extrude_annotations(pathname)

In [73]:
pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
extrude_annotations(pathname)
pathname = "../../Data/TestingData/BAMC-PTXSliding"
extrude_annotations(pathname)

Processing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear\image_417221672548_CLEAN.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_417221672548_CLEAN.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear\image_426794579576_CLEAN.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_426794579576_CLEAN.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear\image_3925135436261_clean.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear/image_3925135436261_clean.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear\image_3929217595322_clean.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear/image_3929217595322_clean.extruded-overlay.mha
